In [13]:
import json
import itertools
data = []

data_type = "dev"
with open(f"{data_type}.jsonl", "r") as f:
    for line in f:
        data.append(json.loads(line))
print("Number of examples:", len(data))

Number of examples: 141


In [2]:
from vncorenlp import VnCoreNLP
segmentor = VnCoreNLP("../../../../VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg")

Entity format:

```
<entity id> <ner_tag> <start_pos> <end_pos> <text>
```

Ví dụ:
```
T3-7	GPE 522 531	community
```

In [3]:
class Entity():
    global_id = 1
    def __init__(self, ner_type, start_pos, end_pos, text):
        self.id = f"T-{Entity.global_id}"
        self.ner_type = ner_type
        self.start_pos = start_pos
        self.end_pos = end_pos
        self.text = text
        Entity.global_id += 1
        
    def __repr__(self):
        return f"{self.id}\t{self.ner_type} {self.start_pos} {self.end_pos}\t{self.text}"
        
class Relation():
    global_id = 1
    def __init__(self, relation_type, *args):
        self.id = f"R-{Relation.global_id}"
        self.relation_type = relation_type
        self.args = args
        Relation.global_id += 1
        
    def __repr__(self):
        res = f"{self.id}\t{self.relation_type}"
        for args_id, entity in enumerate(self.args):
            res += f" Arg{args_id + 1}:{entity.id}"
        return res

In [4]:
from bs4 import BeautifulSoup

def segmentize(text):
    text = segmentor.tokenize(text.strip())
    return " ".join(" ".join(x) for x in text)

def parse_paragraph(data):
    event_texts = {0: ""}
    event_paddings = {0: 0}
    event_ids = []
    e_id = None
    content = ""
    
    for html in data["html_annotation"]:
        for e in BeautifulSoup(html, "html.parser"):
            try:
                e_id = int(e["event_id"])
                event_ids.append(e_id)
                event_texts[e_id] = ""
            except: pass
            if e_id is None:
                continue
            event_texts[e_id] += segmentize(e.text) + " "
    event_ids.sort()
    
    for i, event in enumerate(event_ids):
        cur_event_ids = event_ids[i]
        pre_event_ids = event_ids[i - 1] if i != 0 else 0
        event_paddings[cur_event_ids] = event_paddings[pre_event_ids] + len(event_texts[pre_event_ids])
        content += event_texts[cur_event_ids]
    
    event_texts["0"] = content
    title = segmentize(data["original_doc"]["_source"]["description"])
    return {
        "title": title,
        "content": content,
        "event_texts": event_texts,
        "event_paddings": event_paddings,
    }

In [5]:
def is_match(team1, team2):
    team1 = team1.strip()
    team2 = team2.strip()
    return team1 in team2 or team2 in team1

In [6]:
def full_text_token(text, start, passage):
    end = start + len(text)
    while end < len(passage) and passage[end] != " ":
        end += 1
    while start > 0 and passage[start - 1] != " ":
        start -= 1
    text = passage[start : end]
    return start, text

In [7]:
def _parse_entities_other(ner_type, text, ref_event_ids):
    def parse_entity_other(text, eid):
        if text == "":
            return Entity(ner_type, -1, -1, text)
        
        text = segmentize(text)
        pos = event_texts[eid].lower().find(text.lower())
        if pos != -1:
            pos, text = full_text_token(text, pos, event_texts[eid])
            start_pos = pos + event_paddings[eid]
            end_pos = start_pos + len(text)
            return Entity(ner_type, start_pos, end_pos, text)

        edited_texts = []
        if ner_type == "PER" and len(text.split()) != 1:
            edited_texts = text.split()
        elif ner_type == "TME" and "'" in text:
            text = text.replace("'", "")
            edited_texts = ["phút thứ " + text, "phút " + text] 
        for text in edited_texts:
            edited_ner = parse_entity_other(text, eid)
            if edited_ner.start_pos != edited_ner.end_pos:
                return edited_ner

        return Entity(ner_type, -1, -1, text)

    res = []
    for eid in ref_event_ids.split(","):
        entity = parse_entity_other(text, int(eid))
        if entity.start_pos == entity.end_pos or not entity.text:
            continue
        res.append(entity)
    return res


def _parse_entities_score(ner_type, score1, score2, ref_event_ids):
    def parse_entity_score(eid):
        edited_scores = [score1 + "-" + score2, score1 + " - " + score2, score2 + "-" + score1, score2 + " - " + score1]        
        for score in edited_scores:
            score = segmentize(score)
            pos = event_texts[eid].find(score)
            if pos != -1:
                pos, score = full_text_token(score, pos, event_texts[eid])
                start_pos = event_paddings[eid] + pos
                end_pos = start_pos + len(score)
                return Entity(ner_type, start_pos, end_pos, score)
        return Entity(ner_type, -1, -1, score1 + "-" + score2)

    res = []
    for eid in ref_event_ids.split(","):
        entity = parse_entity_score(int(eid))
        if entity.start_pos == entity.end_pos:
            continue
        res.append(entity)
    return res


def _parse_relations(rel_type, args):
    res = []
    for entities in itertools.product(*args):
        res.append(Relation(rel_type, *entities))
    return res

In [8]:
def parse_entity_relation(data, event_texts, event_paddings):
    entities, relations = [], []
    id = data["train_id"]
    summary = data["match_summary"]
    team = summary["players"]
    score_board = summary["score_board"]
    score_list = summary["score_list"]
    card_list = summary["card_list"]
    subst_list = summary["substitution_list"]
    
    # Yield teams' name
    team1_entities = _parse_entities_other("CLU", team["team1"], team["ref_event_ids"])
    team2_entities = _parse_entities_other("CLU", team["team2"], team["ref_event_ids"])
    entities.extend(team1_entities + team2_entities)
    relations.extend(_parse_relations("COMP", [team1_entities, team2_entities]))
    
    # Yield scores
    score_entities = _parse_entities_score("SCO", score_board["score1"], score_board["score2"], score_board["ref_event_ids"])
    entities.extend(score_entities)
    if int(score_board["score1"]) > int(score_board["score2"]):
        relations.extend(_parse_relations("DEFE", [team1_entities, team2_entities]))
    elif int(score_board["score1"]) < int(score_board["score2"]):
        relations.extend(_parse_relations("DEFE", [team2_entities, team1_entities]))
    
    # Yield list
    for info in score_list:
        player_entities = _parse_entities_other("PER", info["player_name"], info["ref_event_ids"])
        time_entities = _parse_entities_other("TME", info["time"], info["ref_event_ids"])
        team_entities = _parse_entities_other("CLU", info["team"], info["ref_event_ids"])
        # team = team1_entities if info["team"] == team["team1"] else team2_ner
        entities.extend(player_entities + time_entities + team_entities)
        relations.extend(_parse_relations("SCOP", [team_entities, player_entities]))
        relations.extend(_parse_relations("SCOT", [player_entities, time_entities]))       
        
    for info in card_list:
        player_entities = _parse_entities_other("PER", info["player_name"], info["ref_event_ids"])
        time_entities = _parse_entities_other("TME", info["time"], info["ref_event_ids"])
        team_entities = _parse_entities_other("CLU", info["team"], info["ref_event_ids"])
        # team_entites = team1_ner if is_match(info["team"], team1_ner.text) else team2_ner
        entities.extend(player_entities + time_entities + team_entities)
        relations.extend(_parse_relations("CARP", [team_entities, player_entities]))
        relations.extend(_parse_relations("CART", [player_entities, time_entities]))   
        
    for info in subst_list:
        playerin_entities = _parse_entities_other("PER", info["player_in"], info["ref_event_ids"])
        playerout_entities = _parse_entities_other("PER", info["player_out"], info["ref_event_ids"])
        time_entities = [] if "time" not in info else _parse_entities_other("TME", info["time"], info["ref_event_ids"])
        entities.extend(playerin_entities + playerout_entities + time_entities)
        relations.extend(_parse_relations("SUBP", [playerout_entities, playerin_entities]))
        relations.extend(_parse_relations("SUBT", [playerin_entities, time_entities]))
        
    return entities, relations

In [9]:
data[361]["html_annotation"]

['<span>Arsenal chịu áp lực lớn bởi cần chiến thắng để nuôi hy vọng đuổi kịp MU trong cuộc đua vào top 4.</span>',
 '<span class="tag match_result" data=":match_result:" event_id="2">Đội bóng của huấn luyện viên Wenger trải qua 90 phút vất vả nhưng thành công khi thu về 3 điểm trước Middlesbrough sau chiến thắng 2-1.</span>',
 '<span>"Pháo thủ" bước vào trận đấu với áp lực quá lớn. Các cầu thủ đội khách sẵn sàng phạm lỗi trong những pha tranh bóng. </span><span class="tag card_info" data=":card_info:" event_id="3">Điển hình là Chamberlain phải nhận thẻ vàng ngay phút thứ 4.</span>',
 '<span class="tag goal_info" data=":goal_info:" event_id="4">Arsenal chỉ khai thông bế tắc nhờ pha đá phạt hiểm hóc ghi bàn của Alexis Sanchez ở phút 42.</span>',
 '<span>Tiền đạo người Chile thêm lần nữa chứng tỏ đóng góp công lớn vào kết quả thi đấu của đội chủ sân Emirates.</span>',
 '<span class="tag goal_info" data=":goal_info:" event_id="6">Tuy nhiên, đội bóng của huấn luyện viên Wenger tiếp tục gặp 

In [11]:
sample_idx = 0
paragraph = parse_paragraph(data[sample_idx])
content, event_texts, event_paddings = paragraph["content"], paragraph["event_texts"], paragraph["event_paddings"]
entities, relations = parse_entity_relation(data[sample_idx], event_texts, event_paddings)

In [14]:
from tqdm import tqdm
if data_type == "train":
    data[83]["match_summary"]["score_list"][1]["ref_event_ids"] = "4"
    data[361]["match_summary"]["score_list"][0]["ref_event_ids"] = "4"
    data[346]["match_summary"]["card_list"] = []
    data[375]["match_summary"]["score_list"][1]["ref_event_ids"] = "3"
    
for x in tqdm(data):
    file_id = x["train_id"]
    paragraph = parse_paragraph(x)
    content, event_texts, event_paddings = paragraph["content"], paragraph["event_texts"], paragraph["event_paddings"]
    entities, relations = parse_entity_relation(x, event_texts, event_paddings)
    
    with open(f"./{data_type}/{file_id}.txt", "w") as f:
        f.write(paragraph["content"])
    
    with open(f"./{data_type}/{file_id}.ann", "w") as f:
        for entity in entities:
            f.write(str(entity) + "\n")
        for relation in relations:
            f.write(str(relation) + "\n")

100%|██████████| 141/141 [00:20<00:00,  6.78it/s]
